# 유튜브 인기동영상 Data

In [2]:
import pandas as pd

# URL 형식으로 데이터 호출
dataUrl = 'https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/youtube.csv'
data = pd.read_csv(dataUrl,index_col = 0)
data.head()

,title,channelTitle,categoryId,view_count,likes,dislikes,comment_count,channelId,trending_date2
0,[신병] 물자창고,장삐쭈,23,1893473,38249,730,8595,UChbE5OZQ6dRHECsX0tEPEZQ,2021-01-01
1,RAIN(비) - 나로 바꾸자 Switch to me (duet with JYP) MV,RAIN's Official Channel,10,2600864,0,0,20129,UCxXgIeE5hxWxHG6dz9Scg2w,2021-01-01
2,2020년 제야의 종 온라인 타종행사 | 보신각 현장 행사는 진행하지 않습니다.,서울시 · Seoul,29,347049,3564,120,178,UCZUPZW5idAxYp-Asj__lVAA,2021-01-01
3,고기남자의 칠면조 파티,고기남자 MeatMan,26,528458,15372,280,3470,UCT3CumbFIJiW33uq0UI3zlg,2021-01-01
4,골목 3mc를 분노하게 만든 마음고생이 심했을 공릉 백반집 사장님의 푸념?! [예능...,스브스밥집,24,494904,3918,111,3142,UCdWgRSfttvDucq4ApcCg5Mw,2021-01-01


In [13]:
# 인기동영상 제작횟수 많은 채널 상위 10개 추출

data[data['channelId'].isin(data['channelId'].value_counts().head(10).index)].channelTitle.unique()

# data내에 channelId 값이 상위 10개의 id 값에 있는 값만 출력후, title 값으로 출력

array(['장삐쭈', '총몇명', '파뿌리', '짤툰', '런닝맨 - 스브스 공식 채널',
       '엠뚜루마뚜루 : MBC 공식 종합 채널', 'SPOTV', '채널 십오야', '이과장', 'BANGTANTV'],
      dtype=object)

In [17]:
# dislike수가 likes 수보다 높은 동영상 출력

data[data['dislikes']>data['likes']]['channelTitle'].unique()

array(['핫도그TV', 'ASMR 애정TV', '하얀트리HayanTree', '양팡 YangPang',
       '철구형 (CHULTUBE)', '왜냐맨하우스', '(MUTUBE)와꾸대장봉준', '오메킴TV', '육지담',
       'MapleStory_KR', 'ROAD FIGHTING CHAMPIONSHIP', '사나이 김기훈',
       '나혼자산다 STUDIO', 'Gen.G esports'], dtype=object)

In [36]:
# 채널명을 바꾼 케이스 ( channelId는 고유값)

data[['channelTitle','channelId']].duplicated().value_counts() # True 58556개 / False = 1843
chg = data[['channelTitle','channelId']].drop_duplicates().channelId.value_counts() 
print(len(chg[chg>1]))

71


In [45]:
# 특정요일에 인기있던 영상 카테고리

data.loc[:,'days']=pd.to_datetime(data['trending_date2']).dt.day_name() # 날짜형식으로 변환 후 요일별 이름 생성
data[data['days']=='Sunday']['categoryId'].value_counts().index[0] # 일요일에 해당하는 categoryId 값 카운트 후 호출

24

In [55]:
# 요일별 category 개수 출력

data.groupby(['categoryId','days']).size().unstack() # 데이터 프레임 형식으로 출력

days,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
categoryId,,,,,,,
1,243,263,255,274,246,257,234
2,120,105,119,99,128,119,129
10,833,837,776,830,890,894,917
15,187,215,198,217,207,208,207
17,633,668,592,636,682,708,706
19,90,92,87,91,92,89,85
20,283,298,296,289,282,285,291
22,1288,1373,1289,1337,1341,1375,1333
23,568,594,570,556,560,569,566


In [63]:
# viewcount 대비 댓글수가 가장 높은 영상 확인  (view_count값이 0은 제외)

tmp = data[data['view_count']!=0] # view_count 0값 제외
len(tmp[tmp['view_count']==0]) # result = 0

tmp.loc[:,'ratio'] = (tmp['comment_count']/tmp['view_count']).dropna() # viewcount대비 commennt_count 비율 값을 새로운 컬럼 생성
tmp.sort_values('ratio',ascending=False).iloc[0].title # ratio 값 기준으로 내림차순 정렬 후 첫번째 값 title 선택

C:\Users\SuperUser\AppData\Local\Temp\ipykernel_8840\2992716606.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp.loc[:,'ratio'] = (tmp['comment_count']/tmp['view_count']).dropna() # viewcount대비 commennt_count 비율 값을 새로운 컬럼 생성


'60분 동안 댓글이 달리지 않으면, 영상이 삭제됩니다. (챌린지)'

In [68]:
# viewcount 대비 댓글수가 가장 낮은 영상 확인  (view_count,ratio값이 0은 제외)
len(tmp[tmp['ratio']==0]) # result = 777
tmp = tmp[tmp['ratio']!=0] # 0 값 제거
len(tmp[tmp['ratio']==0]) # result = 0

tmp.sort_values('ratio',ascending=True).iloc[0].title # 가장 ratio값이 낮은 title 출력

'Join the BTS #PermissiontoDance Challenge only on YouTube #Shorts'

In [76]:
# like 대비 dislike 수가 가작 적은 영상(0 값은 제외)

len(data[(data['likes']==0)|(data['dislikes']==0)]) # result = 706
tmp = data[(data['likes']!=0)&(data['dislikes']!=0)] # 0값 제외
len(tmp[(tmp['likes']==0)|(tmp['dislikes']==0)]) # result = 0

tmp.loc[:,'ratio'] = (tmp['dislikes']/tmp['likes']).dropna()
tmp.sort_values('ratio',ascending=True).iloc[0].title

C:\Users\SuperUser\AppData\Local\Temp\ipykernel_8840\2299923241.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp.loc[:,'ratio'] = (tmp['dislikes']/tmp['likes']).dropna()


'[줌터뷰] *최초공개* 사부작즈🐰🐶의 비공식 이름은 아이라인즈? 꿀조합 티키타카 가득한 NCT 127 도영&정우의 줌터뷰'

In [92]:
# 가장많은 트랜드 영상 제작한 채널 이름

# data[data['channelId']=='UChbE5OZQ6dRHECsX0tEPEZQ']
data[['channelId','channelTitle']].value_counts().index[0][1] 

'짤툰'

In [98]:
# 20회 이상 인기동영상 리스트에 포함된 동영상

(data[['title','channelId']].value_counts()>=20).sum()

40

# 유튜브 공범컨텐츠 동영상 데이터

In [215]:
import pandas as pd


channel =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/channelInfo.csv')
video =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/videoInfo.csv')
display(channel.head())
display(video.head())

,channelid,subcnt,viewcnt,videocnt,ct,channelname
0,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,410238653,736,2021-09-30 03:01:03,꽈뚜룹
1,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,412531322,736,2021-09-30 09:01:03,꽈뚜룹
2,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,412531322,735,2021-09-30 15:01:03,꽈뚜룹
3,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,412531322,737,2021-09-30 21:01:03,꽈뚜룹
4,UCkQCwnkQfgSuPTTnw_Y7v7w,1320000,412531322,737,2021-10-01 03:01:04,꽈뚜룹


,videopk,viewcnt,likecnt,dislikecnt,favoritecnt,cmcnt,ct,videoname
0,c5JQp6xafqc,1667010,30474,706,0,6587,2021-10-10 15:20:03,공범 EP1
1,c5JQp6xafqc,1669089,30495,707,0,6589,2021-10-10 15:30:03,공범 EP1
2,c5JQp6xafqc,1674759,30522,711,0,6596,2021-10-10 15:40:02,공범 EP1
3,c5JQp6xafqc,1677026,30555,712,0,6604,2021-10-10 15:50:03,공범 EP1
4,c5JQp6xafqc,1681824,30585,713,0,6600,2021-10-10 16:00:03,공범 EP1


In [216]:
# 날짜타입으로 변겨 후 video 데이터에서 videoname의 각 value 값 확인

channel['ct'] = pd.to_datetime(channel['ct'])
video['ct'] = pd.to_datetime(video['ct'])

video['videoname'].value_counts()

 공범 EP1    3492
 공범 EP2    3204
 공범 EP3    2568
 공범 EP4    2280
 공범 EP5    1562
 공범 EP6    1274
 공범 EP7     555
 공범 EP8     266
Name: videoname, dtype: int64

In [217]:
# 수집된 각 video의 가장 최신화 된 날짜의 viewcount 값 출력

tmp = video.drop_duplicates('videoname',keep='last')
tmp[['viewcnt','videoname','ct']].sort_values(['videoname','ct']).reset_index(drop=True)

,viewcnt,videoname,ct
0,3180532,공범 EP1,2021-11-01 15:30:03
1,2199328,공범 EP2,2021-11-01 15:30:03
2,1671294,공범 EP3,2021-11-01 15:30:03
3,1818493,공범 EP4,2021-11-01 15:30:03
4,1503435,공범 EP5,2021-11-01 15:30:04
5,1750222,공범 EP6,2021-11-01 15:30:04
6,1630200,공범 EP7,2021-11-01 15:30:05
7,1289088,공범 EP8,2021-11-01 15:30:05


In [218]:
# channel 데이터 중 2021-10-03일 이후 각 채널의 처음 기록 된 구독자 수 구하기

tmp = channel[channel['ct']>=pd.to_datetime('2021-10-03')].drop_duplicates('channelid',keep='first').sort_values(['ct','channelname']).reset_index(drop=True)
tmp[['channelname','subcnt']]

,channelname,subcnt
0,논리왕 전기,922000
1,야전삽짱재,257000
2,츄정ChuJeong,322000
3,김농밀의 농밀한 삶,7520
4,릴펄 Lilpearl,10100
5,와글와글 WagleWagle,55000
6,조나단,215000
7,형사!탐정되다,14900
8,Balming Tiger,54300
9,곽토리 kwak tori,471000


In [219]:
# 각 채널의 202-10-03 03:00:00~2021-11-01 15:00:00까지 구독자 수 증가량 구하기

end = channel[channel['ct'].dt.strftime('%Y-%m-%d %H')=='2021-11-01 15']
start = channel[channel['ct'].dt.strftime('%Y-%m-%d %H')=='2021-10-03 03']

end_data = end[['channelname','subcnt']].reset_index(drop=True)
start_data = start[['channelname','subcnt']].reset_index(drop=True)

end_data.columns = ['channelname','end_sub']
start_data.columns = ['channelname','start_sub']

m_data = pd.merge(start_data,end_data)
m_data.loc[:,'updown']  = m_data['end_sub']-m_data['start_sub']
m_data[['channelname','updown']]

,channelname,updown
0,꽈뚜룹,70000
1,야전삽짱재,11000
2,츄정ChuJeong,1000
3,논리왕 전기,-11000
4,와글와글 WagleWagle,0
5,조나단,12000
6,형사!탐정되다,10300
7,김농밀의 농밀한 삶,1540
8,릴펄 Lilpearl,11000
9,곽토리 kwak tori,-2000


In [220]:
# 수집간격 5분이하 20분 이상인 데이터 추출 ( 왜 실패??!!!)

tmp = video[video['videoname']==' 공범 EP1'].sort_values('ct').reset_index(drop=True)
tmp[(tmp['ct'].diff(1)>=datetime.timedelta(minutes=20))|(tmp['ct'].diff(1)<=datetime.timedelta(minutes=5))]

,videopk,viewcnt,likecnt,dislikecnt,favoritecnt,cmcnt,ct,videoname
721,c5JQp6xafqc,2228250,34559,849,0,7191,2021-10-13 09:41:37,공범 EP1
722,c5JQp6xafqc,2228250,34559,849,0,7191,2021-10-13 09:41:37,공범 EP1
1636,c5JQp6xafqc,2707933,37901,988,0,7410,2021-10-19 18:20:03,공범 EP1


In [221]:
# 각 에피소드 별 시작 날짜 및 이름 묶어서 데이터프레임 출력

tmp = video[['videoname','ct']].sort_values(['ct','videoname']).drop_duplicates('videoname',keep='first').reset_index(drop=True) # 전처리 순서 집중하기
tmp['date']=tmp['ct'].dt.date # dt.date = > 'YYYY-MM-DD' 형식으로 출력(문자열)
tmp[['videoname','date']]

,videoname,date
0,공범 EP1,2021-10-07
1,공범 EP2,2021-10-09
2,공범 EP3,2021-10-14
3,공범 EP4,2021-10-16
4,공범 EP5,2021-10-21
5,공범 EP6,2021-10-23
6,공범 EP7,2021-10-28
7,공범 EP8,2021-10-30


In [222]:
#“공범” 컨텐츠의 경우 19:00시에 공개 되는것으로 알려져있다. 공개된 날의 21시의 viewcnt, ct, videoname 으로 구성된 데이터 프레임을 
#viewcnt를 내림차순으로 정렬하여 출력

tmp = video[video['ct'].dt.hour==21].sort_values(['ct','videoname']).drop_duplicates('videoname',keep='first')
tmp[['videoname','viewcnt','ct']].sort_values('viewcnt',ascending=False).reset_index(drop=True)

,videoname,viewcnt,ct
0,공범 EP8,264029,2021-10-30 21:00:08
1,공범 EP7,252032,2021-10-28 21:00:04
2,공범 EP4,217674,2021-10-16 21:00:04
3,공범 EP6,213899,2021-10-23 21:00:06
4,공범 EP5,201179,2021-10-21 21:00:04
5,공범 EP2,148144,2021-10-09 21:00:03
6,공범 EP3,147183,2021-10-14 21:00:04
7,공범 EP1,117340,2021-10-07 21:00:03


In [223]:
# video 정보의 가장 최근 데이터들에서 각 에피소드의 싫어요/좋아요 비율을 ratio 컬럼으로 만들고 
# videoname, ratio로 구성된 데이터 프레임을 ratio를 오름차순으로 정렬

tmp = video.sort_values(['ct','videoname'],ascending=False).drop_duplicates('videoname',keep='first')
tmp['ratio'] = (tmp['dislikecnt']/tmp['likecnt']).dropna()

tmp[['videoname','ratio']].sort_values('ratio',ascending=True).reset_index(drop=True)

,videoname,ratio
0,공범 EP7,0.013636
1,공범 EP6,0.015881
2,공범 EP4,0.017971
3,공범 EP5,0.021128
4,공범 EP3,0.022360
5,공범 EP1,0.027118
6,공범 EP2,0.029844
7,공범 EP8,0.141357


In [224]:
#2021-11-01 00:00:00 ~ 15:00:00까지 각 에피소드별 viewcnt의 증가량을 데이터 프레임 출력
def check(x):
    result = max(x) - min(x)
    return result

tmp = video[(video['ct']>=pd.to_datetime('2021-11-01 00:00:00'))&(video['ct']<=pd.to_datetime('2021-11-01 15:00:00'))].reset_index(drop=True)
tmp[['videoname','viewcnt']].groupby('videoname').agg(check)

,viewcnt
videoname,
공범 EP1,13298
공범 EP2,10300
공범 EP3,9927
공범 EP4,9824
공범 EP5,10824
공범 EP6,14141
공범 EP7,26949
공범 EP8,89147


In [228]:
# video 데이터 중에서 중복되는 데이터가 존재한다. 중복되는 각 데이터의 시간대와 videoname 출력

tmp = video[(video.duplicated())==True]
tmp[['videoname','ct']]

,videoname,ct
722,공범 EP1,2021-10-13 09:41:37
3927,공범 EP2,2021-10-13 09:41:37


# 월드컵 출전선수 골기록

In [321]:
data= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/worldcup/worldcupgoals.csv')
data.head()

,Player,Goals,Years,Country
0,Miroslav Klose,16,2002-2006-2010-2014,Germany
1,Ronaldo,15,1998-2002-2006,Brazil
2,Gerd Muller,14,1970-1974,Germany
3,Just Fontaine,13,1958,France
4,Pele,12,1958-1962-1966-1970,Brazil


In [322]:
# 주어진 전체 기간의 각 나라별 골득점수 상위 5개 국가와 득점수 데이터프레임 형태 출력

data.groupby('Country').sum().sort_values('Goals',ascending=False).iloc[:5]

,Goals
Country,
Brazil,228
Germany,226
Argentina,135
Italy,123
France,115


In [323]:
# 주어진 전체기간동안 골득점을 한 선수가 가장 많은 나라 상위 5개 국가와 그 선수 숫자를 데이터 프레임 형식으로 출력하라

tmp = data.groupby('Country')['Player'].size().sort_values(ascending=False).head(5).to_frame()
tmp.columns=['cnt']
tmp

,cnt
Country,
Brazil,81
Germany,78
Italy,60
Argentina,59
France,58


In [324]:
# Years 컬럼은 년도 -년도 형식으로 구성되어있고, 각 년도는 4자리 숫자이다. 
#년도 표기가 4자리 숫자로 안된 케이스가 존재한다. 해당 건은 몇건인지 출력하라
cnt=0
for i in range(len(data['Years'].str.split('-'))):
    for j in range(len(data['Years'].str.split('-')[i])):
        if len(data['Years'].str.split('-')[i][j]) !=4:
            cnt+=1
print(cnt)

45


In [335]:
# Q23에서 발생한 예외 케이스를 제외한 데이터프레임을 df2라고 정의하고 데이터의 행의 숫자를 출력

data['yearLst'] = data.Years.str.split('-')

def checkFour(x):
    for value in x:
        if len(str(value)) != 4:
            return False
        
    return True
    
df2 = data[data.check ==True].reset_index(drop=True)
print(df2.shape[0])

1250


In [350]:
df2

,Player,Goals,Years,Country,check,yearLst,LenCup
0,Miroslav Klose,16,2002-2006-2010-2014,Germany,True,"[2002, 2006, 2010, 2014]",4
1,Ronaldo,15,1998-2002-2006,Brazil,True,"[1998, 2002, 2006]",3
2,Gerd Muller,14,1970-1974,Germany,True,"[1970, 1974]",2
3,Just Fontaine,13,1958,France,True,[1958],1
4,Pele,12,1958-1962-1966-1970,Brazil,True,"[1958, 1962, 1966, 1970]",4
...,...,...,...,...,...,...,...
1245,Josip Skoblar,1,1962,Yugoslavia,True,[1962],1
1246,Safet Susic,1,1982-1990,Yugoslavia,True,"[1982, 1990]",2
1247,Aleksandar Tirnanic,1,1930,Yugoslavia,True,[1930],1
1248,Djordje Vujadinovic,1,1930,Yugoslavia,True,[1930],1


In [342]:
# 월드컵 출전횟수를 나타내는 ‘LenCup’ 컬럼을 추가하고 4회 출전한 선수의 숫자를 구하여라

df2['LenCup']=df2['yearLst'].str.len()
result = df2['LenCup'].value_counts()[4]
result

16

In [355]:
# Yugoslavia 국가의 월드컵 출전횟수가 2회인 선수들의 숫자를 구하여라
len(df2[(df2['Country']=='Yugoslavia')&(df2['LenCup']==2)])

7

In [363]:
#2002년도에 출전한 전체 선수는 몇명인가?

len(df2[df2['Years'].str.contains('2002')])

156

In [373]:
# 이름에 ‘carlos’ 단어가 들어가는 선수의 숫자는 몇 명인가? (대, 소문자 구분 x)

len(df2[df2['Player'].str.lower().str.contains('carlos')])

13

In [379]:
#월드컵 출전 횟수가 1회뿐인 선수들 중에서 가장 많은 득점을 올렸던 선수는 누구인가?

df2[['Player','Goals']][df2['LenCup']==1].iloc[0].Player

'Just Fontaine'

In [385]:
#월드컵 출전횟수가 1회 뿐인 선수들이 가장 많은 국가는 어디인가?

df2[df2['LenCup']==1]['Country'].value_counts().index[0]

'Brazil'

# 따릉이 데이터

In [3]:
import pandas as pd
data =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bicycle/seoul_bi.csv')
data.head()

,대여일자,대여시간,대여소번호,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리,사용시간
0,2021-06-01,0,3541,정기권,F,~10대,1,0.00,0.00,0.00,8
1,2021-06-01,0,765,정기권,F,~10대,1,27.21,0.35,1526.81,19
2,2021-06-01,0,2637,정기권,F,~10대,1,41.40,0.37,1608.56,18
3,2021-06-01,0,2919,정기권,F,~10대,1,0.00,0.00,0.00,75
4,2021-06-01,0,549,정기권,F,~10대,1,13.04,0.17,731.55,6


In [39]:
# 대여일자별 데이터의 수를 데이터프레임으로 출력하고, 가장 많은 데이터가 있는 날짜 출력

rst = data['대여일자'].value_counts().to_frame().reset_index()
rst.columns = ['대여일자','cnt']
ans = rst[rst['cnt']==rst['cnt'].max()]['대여일자'][0]
display(rst)
print(ans)

,대여일자,cnt
0,2021-06-04,56788
1,2021-06-02,56100
2,2021-06-08,55623
3,2021-06-14,54945
4,2021-06-09,54507
5,2021-06-05,52730
6,2021-06-07,52623
7,2021-06-11,52089
8,2021-06-12,48569
9,2021-06-01,48081


2021-06-04


In [50]:
# 각 일자의 요일을 표기하고 (‘Monday’ ~’Sunday’) ‘day_name’컬럼을 추가
#이를 이용하여 각 요일별 이용 횟수의 총합을 데이터 프레임으로 출력


data['day_name']=pd.to_datetime(data['대여일자']).dt.day_name()
tmp=data.groupby('day_name').size().to_frame().reset_index()
tmp.columns=['day','cnt']
tmp.sort_values('cnt',ascending=False)

,day,cnt
6,Wednesday,110607
0,Friday,108877
1,Monday,107568
5,Tuesday,103704
2,Saturday,101299
3,Sunday,90918
4,Thursday,55977


In [78]:
# 각 요일별 가장 많이 이용한 대여소의 이용횟수와 대여소 번호를 데이터 프레임으로 출력하라
# data.head()
data.groupby(['day_name','대여소번호'],as_index=False).size().sort_values(['day_name','size'],ascending=False)\ #코드가 길어서 \사용
.drop_duplicates('day_name',keep='first').reset_index(drop=True)

,day_name,대여소번호,size
0,Wednesday,502,282
1,Tuesday,502,267
2,Thursday,2715,137
3,Sunday,502,372
4,Saturday,502,378
5,Monday,502,242
6,Friday,502,277


In [110]:
# 나이대별 대여구분 코드의 (일일권/전체횟수) 비율을 구한 후 가장 높은 비율을 가지는 나이대를 확인하라
#일일권의 경우 일일권 과 일일권(비회원)을 모두 포함하라

a = data[data['대여구분코드'].str.contains('일일권')]['연령대코드'].value_counts().sort_index() # 일일권, 일일권(비회원) 포함한 연령별 합계
b = data['연령대코드'].value_counts().sort_index() # 연령별 총 합계
ratio = (a/b).sort_values(ascending=False)
print(ratio.index[0])

~10대


In [116]:
# 연령대별 평균 이동거리를 구하여라
data.groupby('연령대코드')['이동거리'].mean().to_frame(name='mean_dis')

,mean_dis
연령대코드,
20대,3211.890552
30대,3341.443859
40대,3514.857416
50대,3593.668100
60대,3538.145737
70대~,3085.039641
~10대,2634.426279


In [120]:
data['대여일자']=pd.to_datetime(data['대여일자'])

In [151]:
# 연령대 코드가 20대인 데이터를 추출하고,이동거리값이 추출한 데이터의 이동거리값의 평균 이상인 데이터를 추출한다.
# 최종 추출된 데이터를 대여일자, 대여소 번호 순서로 내림차순 정렬 후 1행부터 200행까지의 탄소량의 평균을 소숫점 3째 자리까지 구하여라

# tmp = data[(data['연령대코드']=='20대')&(data['이동거리']>=data['이동거리'].mean())].sort_values(['대여일자','대여소번호'],ascending=False)\
# .reset_index(drop=True).iloc[:200] --> 오답인 이유 : 20대 데이터 "추출 후" 이동거리 평균에 따른 추출인데 난 20대이면서, 평균에 따른 데이터 추출

tw = data[data.연령대코드 =='20대'].reset_index(drop=True)
tw_mean = tw[tw.이동거리 >= tw.이동거리.mean()].reset_index(drop=True)
tw_mean['탄소량'] =tw_mean['탄소량'].astype('float')
target =tw_mean.sort_values(['대여일자','대여소번호'], ascending=False).reset_index(drop=True).iloc[:200].탄소량
result = round(target.sum()/len(target),3)
print(result)


1.613


In [173]:
#6월 7일 ~10대의 “이용건수”의 중앙값은?

data[(data['연령대코드']=='~10대')&(data['대여일자']==pd.to_datetime('2021-06-07'))]['이용건수'].median()

1.0

In [206]:
#평일 (월~금) 출근 시간대(오전 6,7,8시)의 대여소별 이용 횟수를 구해서 데이터 프레임 형태로 표현한 후 
#각 대여시간별 이용 횟수의 상위 3개 대여소와 이용횟수를 출력하라
tmp = data[(data['대여일자'].dt.weekday<5)&(data['대여시간'].isin([6,7,8]))].groupby(['대여시간','대여소번호'])\
.size().to_frame('이용횟수')
ans = tmp.sort_values(['대여시간','이용횟수'],ascending=False).groupby('대여시간').head(3)
ans

이용횟수
대여시간 대여소번호      
8    2701    119
     646     115
     1152     92
7    259     104
     230      77
     726      77
6    2744     45
     1125     40
     1028     36

In [209]:
#이동거리의 평균 이상의 이동거리 값을 가지는 데이터를 추출하여 추출데이터의 이동거리의 표본표준편차 값을 구하여라

data[data['이동거리']>=data['이동거리'].mean()]['이동거리'].std()

5092.139707505305

In [214]:
# 남성(‘M’ or ‘m’)과 여성(‘F’ or ‘f’)의 이동거리값의 평균값을 구하여라

m = data[data['성별'].isin(['M','m'])]['이동거리'].mean()
f = data[data['성별'].isin(['F','f'])]['이동거리'].mean()
ans = pd.DataFrame({'성별':['남','여'],'이동거리':[m,f]})
ans

,성별,이동거리
0,남,3209.110871
1,여,3468.575025


# 전 세계 행복도 지표 데이터

In [216]:
import pandas as pd
data =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/happy2/happiness.csv',encoding='utf-8')
data.head()

,행복랭킹,나라명,점수,상대GDP,사회적지원,행복기대치,선택의 자유도,관대함,부패에 대한인식,년도
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393,2019
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410,2019
2,3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341,2019
3,4,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118,2019
4,5,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298,2019


In [233]:
#데이터는 2018년도와 2019년도의 전세계 행복 지수를 표현한다. 각년도의 행복랭킹 10위를 차지한 나라의 행복점수의 평균을 구하여라

data[data['행복랭킹']==10]['점수'].mean()

7.259

In [236]:
# 데이터는 2018년도와 2019년도의 전세계 행복 지수를 표현한다. 
# 각년도의 행복랭킹 50위이내의 나라들의 각각의 행복점수 평균을 데이터프레임으로 표시하라
data[data['행복랭킹']<=50].groupby('년도')['점수'].mean().to_frame(name='평균 행복점수')

,평균 행복점수
년도,
2018,6.64678
2019,6.67002


In [249]:
#2018년도 데이터들만 추출하여 행복점수와 부패에 대한 인식에 대한 상관계수를 구하여라
data[data['년도']==2018][['점수','부패에 대한인식']].corr().iloc[0,1]

0.40529152271510027

In [265]:
# 2018년도와 2019년도의 행복랭킹이 변화하지 않은 나라명의 수를 구하여라
display(len(data[['행복랭킹','나라명']])) # 전체 데이터 수
display(len(data[['행복랭킹','나라명']].drop_duplicates())) # 중복을 제거 한 데이터 (여기서 중복 데이터는 행복랭킹이 변화하지 않은 데이터)
len(data[['행복랭킹','나라명']])- len(data[['행복랭킹','나라명']].drop_duplicates())

312

297

15

In [263]:
data[data[['행복랭킹','나라명']].duplicated()] # 중복된 데이터

,행복랭킹,나라명,점수,상대GDP,사회적지원,행복기대치,선택의 자유도,관대함,부패에 대한인식,년도
156,1,Finland,7.632,1.305,1.592,0.874,0.681,0.202,0.393,2018
159,4,Iceland,7.495,1.343,1.644,0.914,0.677,0.353,0.138,2018
163,8,New Zealand,7.324,1.268,1.601,0.876,0.669,0.365,0.389,2018
177,22,Malta,6.627,1.270,1.525,0.884,0.645,0.376,0.142,2018
189,34,Singapore,6.343,1.529,1.451,1.008,0.631,0.261,0.457,2018
208,53,Latvia,5.933,1.148,1.454,0.671,0.363,0.092,0.066,2018
211,56,Jamaica,5.890,0.819,1.493,0.693,0.575,0.096,0.031,2018
215,60,Kazakhstan,5.790,1.143,1.516,0.631,0.454,0.148,0.121,2018
220,65,Peru,5.663,0.934,1.249,0.674,0.530,0.092,0.034,2018
231,76,Hong Kong,5.430,1.405,1.290,1.030,0.524,0.246,0.291,2018


In [290]:
#2019년도 데이터들만 추출하여 각변수간 상관계수를 구하고 내림차순으로 정렬한 후 상위 5개를 데이터 프레임으로 출력하라. 
#컬럼명은 v1,v2,corr으로 표시하라

tmp = data[data['년도']==2019].corr().iloc[:-1,:-1].unstack().to_frame(name='corr').reset_index()
tmp.columns=['v1','v2','corr']
tmp[tmp['corr']!=1].sort_values('corr',ascending=False).drop_duplicates('corr').head(5).reset_index(drop=True)

,v1,v2,corr
0,행복기대치,상대GDP,0.835462
1,상대GDP,점수,0.793883
2,행복기대치,점수,0.779883
3,사회적지원,점수,0.777058
4,사회적지원,상대GDP,0.754906


In [306]:
# 각 년도별 하위 행복점수의 하위 5개 국가의 평균 행복점수를 구하여라
data.groupby('년도').tail(5).groupby('년도').mean()[['점수']]

,점수
년도,
2018,3.1800
2019,3.1408


In [310]:
# 2019년 데이터를 추출하고 해당데이터의 상대 GDP 평균 이상의 나라들과 
#평균 이하의 나라들의 행복점수 평균을 각각 구하고 그 차이값을 출력하라

a = data[(data['년도']==2019)&(data['상대GDP']>=data['상대GDP'].mean())]['점수'].mean()
b = data[(data['년도']==2019)&(data['상대GDP']<=data['상대GDP'].mean())]['점수'].mean()
a-b

1.3904839572192538

In [337]:
# 각년도의 부패에 대한인식을 내림차순 정렬했을때 상위 20개 국가의 부패에 대한인식의 평균을 구하여라

data[['년도','부패에 대한인식']].sort_values(['년도','부패에 대한인식'],ascending=False).groupby('년도').head(20).\
groupby('년도').mean()

,부패에 대한인식
년도,
2018,0.3267
2019,0.3201


In [349]:
# 2018년도 행복랭킹 50위 이내에 포함됐다가 2019년 50위 밖으로 밀려난 국가의 숫자를 구하여라

len(set(data[(data['년도']==2018)&(data['행복랭킹']<=50)]['나라명'])-set(data[(data['년도']==2019)&(data['행복랭킹']<=50)]['나라명']))

4

# 지역구 에너지 소비량 데이터

In [389]:
import pandas as pd
data= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/consum/Tetuan%20City%20power%20consumption.csv')
data.head()

,DateTime,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,Zone 1 Power Consumption,Zone 2 Power Consumption,Zone 3 Power Consumption
0,1/1/2017 0:00,6.559,73.8,0.083,0.051,0.119,34055.69620,16128.87538,20240.96386
1,1/1/2017 0:10,6.414,74.5,0.083,0.070,0.085,29814.68354,19375.07599,20131.08434
2,1/1/2017 0:20,6.313,74.5,0.080,0.062,0.100,29128.10127,19006.68693,19668.43373
3,1/1/2017 0:30,6.121,75.0,0.083,0.091,0.096,28228.86076,18361.09422,18899.27711
4,1/1/2017 0:40,5.921,75.7,0.081,0.048,0.085,27335.69620,17872.34043,18442.40964


In [390]:
#DateTime컬럼을 통해 각 월별로 몇개의 데이터가 있는지 데이터 프레임으로 구하여라

data['DateTime']=pd.to_datetime(data['DateTime'])
tmp = data
tmp.loc[:,'month']=tmp['DateTime'].dt.month
tmp.groupby('month').size().to_frame(name='cnt')

,cnt
month,
1,4464
2,4032
3,4464
4,4320
5,4464
6,4320
7,4464
8,4464
9,4320


In [398]:
# 3월달의 각 시간대별 온도의 평균들 중 가장 낮은 시간대의 온도를 출력하라

tmp = data[data['DateTime'].dt.month ==3]
tmp.groupby(tmp['DateTime'].dt.hour)['Temperature'].mean().sort_values().values[0]

11.506612903225806

In [399]:
# 3월달의 각 시간대별 온도의 평균들 중 가장 높은 시간대의 온도를 출력하라

tmp = data[data['DateTime'].dt.month ==3]
tmp.groupby(tmp['DateTime'].dt.hour)['Temperature'].mean().sort_values(ascending=False).values[0]

18.393602150537635

In [402]:
data.columns

Index(['DateTime', 'Temperature', 'Humidity', 'Wind Speed',
       'general diffuse flows', 'diffuse flows', 'Zone 1 Power Consumption',
       'Zone 2  Power Consumption', 'Zone 3  Power Consumption', 'month'],
      dtype='object')

In [404]:
# Zone 1 Power Consumption 컬럼의 value값의 크기가 Zone 2 Power Consumption 컬럼의 value값의 크기보다 큰 데이터들의 
# Humidity의 평균을 구하여라

data[data['Zone 1 Power Consumption']>data['Zone 2  Power Consumption']]['Humidity'].mean()

68.23624448055094

In [418]:
# 각 zone의 에너지 소비량의 상관관계를 구해서 데이터 프레임으로 표기하라

data[['Zone 1 Power Consumption','Zone 2  Power Consumption','Zone 3  Power Consumption']].corr()

,Zone 1 Power Consumption,Zone 2 Power Consumption,Zone 3 Power Consumption
Zone 1 Power Consumption,1.000000,0.834519,0.750733
Zone 2 Power Consumption,0.834519,1.000000,0.570932
Zone 3 Power Consumption,0.750733,0.570932,1.000000


In [427]:
#Temperature의 값이 10미만의 경우 A, 10이상 20미만의 경우 B,20이상 30미만의 경우 C,
#그 외의 경우 D라고 할때 각 단계의 데이터 숫자를 구하여라

def chg(x):
    if x<10:
        return 'A'
    elif 10<=x<20:
        return 'B'
    elif 20<=x<30:
        return 'C'
    else:
        return 'D'
    
tmp = data['Temperature'].apply(lambda x: chg(x))
tmp.value_counts()

B    26993
C    21105
A     2874
D     1444
Name: Temperature, dtype: int64

In [430]:
# 6월 데이터중 12시의 Temperature의 표준편차를 구하여라
data[(data['DateTime'].dt.month==6)&(data['DateTime'].dt.hour==12)]['Temperature'].std()

2.049941782795103

In [431]:
# 6월 데이터중 12시의 Temperature의 분산을 구하여라

data[(data['DateTime'].dt.month==6)&(data['DateTime'].dt.hour==12)]['Temperature'].var()

4.202261312849164

In [443]:
# Temperature의 평균이상의 Temperature의 값을 가지는 데이터를 Temperature를 기준으로 정렬 했을때 4번째 행의 Humidity 값은?

data[data['Temperature']>data['Temperature'].mean()].sort_values('Temperature')['Humidity'].values[3]

87.9

In [445]:
#Temperature의 중간값 이상의 Temperature의 값을 가지는 데이터를Temperature를 기준으로 정렬 했을때 4번째 행의 Humidity 값은?
data[data['Temperature']>=data['Temperature'].median()].sort_values('Temperature')['Humidity'].values[3]

80.3

# 포켓몬 정보 데이터

In [446]:
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/pok/Pokemon.csv')
data.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [451]:
#Legendary 컬럼은 전설포켓몬 유무를 나타낸다.전설포켓몬과 그렇지 않은 포켓몬들의 HP평균의 차이를 구하여라

abs(data.groupby('Legendary')['HP'].mean()[0]-data.groupby('Legendary')['HP'].mean()[1])

25.55614861329147

In [454]:
#Type 1은 주속성 Type 2 는 부속성을 나타낸다. 가장 많은 부속성 종류는 무엇인가?

data['Type 2'].value_counts().index[0]

'Flying'

In [459]:
# 가장 많은 Type 1 의 종의 평균 Attack 을 평균 Defense로 나눈값은?

data[data['Type 1']==data['Type 1'].value_counts().index[0]]['Attack'].mean()/data[data['Type 1']==data['Type 1'].value_counts().index[0]]['Defense'].mean()

1.0165238678090576

In [469]:
# 포켓몬 세대(Generation) 중 가장많은 Legendary를 보유한 세대는 몇세대인가?

# 1. groupby 해결법
data[data['Legendary']==True].groupby(['Generation','Legendary']).size().to_frame(name='cnt').sort_values('cnt',ascending=False).index[0][0]

#2 value_counts로 간단하게 해결하기

data[data['Legendary']==True]['Generation'].value_counts().index[0]

3

In [477]:
# HP’, ‘Attack’, ‘Defense’, ‘Sp. Atk’, ‘Sp. Def’, ‘Speed’ 간의 상관 계수중 가장 절댓값이 큰 두 변수와 그 값을 구하여라

tmp = data[['HP','Attack', 'Defense', 'Sp. Atk','Sp. Def','Speed']].corr().unstack().reset_index().rename(columns={0: "corr"})
tmp[tmp['corr']!=1].sort_values('corr',ascending=False).iloc[0]

level_0     Sp. Def
level_1     Defense
corr       0.510747
Name: 26, dtype: object

In [490]:
# 각 Generation의 Attack으로 오름차순 정렬시 상위 3개 데이터들(18개)의 Attack의 전체 평균을 구하여라

data.sort_values(['Generation','Attack']).groupby('Generation').head(3)['Attack'].mean()

19.5

In [491]:
# 각 Generation의 Attack으로 내림차순 정렬시 상위 5개 데이터들(30개)의 Attack의 전체 평균을 구하여라

data.sort_values(['Generation','Attack'],ascending=False).groupby('Generation').head(5)['Attack'].mean()

157.23333333333332

In [499]:
# 가장 흔하게 발견되는 (Type1 , Type2) 의 쌍은 무엇인가?

data[['Type 1','Type 2']].value_counts().head(1)

Type 1  Type 2
Normal  Flying    24
dtype: int64

In [503]:
#한번씩만 존재하는 (Type1 , Type2)의 쌍의 갯수는 몇개인가?

tmp = data[['Type 1','Type 2']].value_counts()
len(tmp[tmp==1])

39